In [40]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [41]:
data_path = 'data/ev_charging_patterns.csv'

In [42]:
try:
    df = pd.read_csv(data_path)
    print("Tải dữ liệu thành công!")
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file dữ liệu tại đường dẫn: {data_path}")
    print("Hãy đảm bảo bạn đã tải file CSV và đặt đúng đường dẫn.")
    # Thoát sớm nếu không có dữ liệu
    # Hoặc bạn có thể xử lý tiếp nếu muốn tạo dữ liệu giả
    df = None

Tải dữ liệu thành công!


In [54]:
df.head()

,User ID,Vehicle Model,Battery Capacity (kWh),Charging Station ID,Charging Station Location,Charging Start Time,Charging End Time,Energy Consumed (kWh),Charging Duration (hours),Charging Rate (kW),Charging Cost (USD),Time of Day,Day of Week,State of Charge (Start %),State of Charge (End %),Distance Driven (since last charge) (km),Temperature (°C),Vehicle Age (years),Charger Type,User Type
0,User_1,BMW i3,108.463007,Station_391,Houston,2024-01-01 00:00:00,2024-01-01 00:39:00,60.712346,0.591363,36.389181,13.087717,Evening,Tuesday,29.371576,86.119962,293.602111,27.947953,2.0,DC Fast Charger,Commuter
1,User_2,Hyundai Kona,100.000000,Station_428,San Francisco,2024-01-01 01:00:00,2024-01-01 03:01:00,12.339275,3.133652,30.677735,21.128448,Morning,Monday,10.115778,84.664344,112.112804,14.311026,3.0,Level 1,Casual Driver
2,User_3,Chevy Bolt,75.000000,Station_181,San Francisco,2024-01-01 02:00:00,2024-01-01 04:48:00,19.128876,2.452653,27.513593,35.667270,Morning,Thursday,6.854604,69.917615,71.799253,21.002002,2.0,Level 2,Commuter
3,User_4,Hyundai Kona,50.000000,Station_327,Houston,2024-01-01 03:00:00,2024-01-01 06:42:00,79.457824,1.266431,32.882870,13.036239,Evening,Saturday,83.120003,99.624328,199.577785,38.316313,1.0,Level 1,Long-Distance Traveler
4,User_5,Hyundai Kona,50.000000,Station_108,Los Angeles,2024-01-01 04:00:00,2024-01-01 05:46:00,19.629104,2.019765,10.215712,10.161471,Morning,Saturday,54.258950,63.743786,203.661847,-7.834199,1.0,Level 1,Long-Distance Traveler


In [43]:
if df is not None:
    print("\nThông tin DataFrame (df.info()):")
    df.info()

    print("\n5 dòng dữ liệu đầu tiên (df.head()):")
    print(df.head())

    print("\nKiểm tra các giá trị duy nhất trong cột mục tiêu (User Type):")
    # Lưu ý: Tên cột có dấu cách 'User Type'
    if 'User Type' in df.columns:
         print(df['User Type'].value_counts())
    else:
        print("Lỗi: Không tìm thấy cột 'User Type'. Hãy kiểm tra lại tên cột.")


Thông tin DataFrame (df.info()):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1320 entries, 0 to 1319
Data columns (total 20 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   User ID                                   1320 non-null   object 
 1   Vehicle Model                             1320 non-null   object 
 2   Battery Capacity (kWh)                    1320 non-null   float64
 3   Charging Station ID                       1320 non-null   object 
 4   Charging Station Location                 1320 non-null   object 
 5   Charging Start Time                       1320 non-null   object 
 6   Charging End Time                         1320 non-null   object 
 7   Energy Consumed (kWh)                     1254 non-null   float64
 8   Charging Duration (hours)                 1320 non-null   float64
 9   Charging Rate (kW)                        1254 non-null   float64
 10  Ch

In [44]:
if df is not None and 'User Type' in df.columns:
    # Xác định biến mục tiêu (target) - Đảm bảo tên cột chính xác
    target = 'User Type'

    # Xác định các cột chắc chắn không dùng làm features (Target và các ID)
    columns_to_drop = [
        target,
        'User ID',             # Mã định danh người dùng
        'Charging Station ID'  # Mã định danh trạm sạc
        # Cân nhắc thêm 'Charging Station Location' nếu có quá nhiều giá trị và khó xử lý
    ]

    # Tạo DataFrame features X và Series target y
    try:
        X = df.drop(columns=columns_to_drop)
        y = df[target]
        print("Đã tách Features (X) và Target (y) thành công.")
        print(f"Số lượng features: {X.shape[1]}")
    except KeyError as e:
        print(f"Lỗi khi drop cột: {e}. Có thể tên cột trong 'columns_to_drop' không tồn tại trong DataFrame.")
        X, y = None, None
else:
    print("Bỏ qua bước chuẩn bị do dữ liệu chưa được tải hoặc thiếu cột Target.")
    X, y = None, None

Đã tách Features (X) và Target (y) thành công.
Số lượng features: 17


In [45]:
if X is not None and y is not None:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.2,    # 20% dữ liệu cho tập test
        random_state=42,  # Để kết quả có thể tái lặp
        stratify=y        # Giữ tỷ lệ phân bố của các lớp trong target ở cả 2 tập
    )
    print("Đã chia dữ liệu thành tập Train và Test:")
    print(f"Kích thước X_train: {X_train.shape}")
    print(f"Kích thước X_test: {X_test.shape}")
    print(f"Kích thước y_train: {y_train.shape}")
    print(f"Kích thước y_test: {y_test.shape}")
else:
    print("Bỏ qua bước chia dữ liệu.")

Đã chia dữ liệu thành tập Train và Test:
Kích thước X_train: (1056, 17)
Kích thước X_test: (264, 17)
Kích thước y_train: (1056,)
Kích thước y_test: (264,)


In [46]:
if 'X_train' in locals(): # Kiểm tra xem X_train đã được tạo chưa
    # Xác định tên các cột số và cột dạng chữ TỪ TẬP TRAIN (tránh data leakage)
    numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X_train.select_dtypes(include=['object', 'category']).columns

    print(f"\nCác cột số (Numeric): {list(numeric_features)}")
    print(f"Các cột chữ/phân loại (Categorical): {list(categorical_features)}")

    # Tạo Pipeline xử lý cho cột số
    numeric_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')), # Điền giá trị thiếu bằng trung vị
        ('scaler', StandardScaler())                   # Chuẩn hóa dữ liệu
    ])

    # Tạo Pipeline xử lý cho cột dạng chữ
    categorical_pipeline = Pipeline(steps=[
        # Nếu cột chữ có thể có NaN, thêm bước impute:
        ('imputer', SimpleImputer(strategy='most_frequent')), # Điền bằng giá trị xuất hiện nhiều nhất
        ('onehot', OneHotEncoder(handle_unknown='ignore')) # Chuyển thành số, bỏ qua giá trị lạ ở tập test
    ])

    # Tạo ColumnTransformer để áp dụng các pipeline tương ứng
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_pipeline, numeric_features),
            ('cat', categorical_pipeline, categorical_features)
        ],
        remainder='passthrough' # Giữ lại các cột không được chỉ định nếu có (an toàn hơn)
    )

    print("\nĐã định nghĩa xong Preprocessor.")

else:
    print("Bỏ qua bước định nghĩa Preprocessor do thiếu dữ liệu Train.")
    preprocessor = None


Các cột số (Numeric): ['Battery Capacity (kWh)', 'Energy Consumed (kWh)', 'Charging Duration (hours)', 'Charging Rate (kW)', 'Charging Cost (USD)', 'State of Charge (Start %)', 'State of Charge (End %)', 'Distance Driven (since last charge) (km)', 'Temperature (°C)', 'Vehicle Age (years)']
Các cột chữ/phân loại (Categorical): ['Vehicle Model', 'Charging Station Location', 'Charging Start Time', 'Charging End Time', 'Time of Day', 'Day of Week', 'Charger Type']

Đã định nghĩa xong Preprocessor.


In [48]:
# Kiểm tra NaN (tùy chọn) - SỬA LẠI
try:
    # Thử chuyển sang dense array để kiểm tra
    print(f"Số NaN trong X_train_processed: {np.isnan(X_train_processed.toarray()).sum()}")
    print(f"Số NaN trong X_test_processed: {np.isnan(X_test_processed.toarray()).sum()}")
except AttributeError:
    # Nếu nó không phải sparse matrix (đã là dense array)
    print(f"Số NaN trong X_train_processed: {np.isnan(X_train_processed).sum()}")
    print(f"Số NaN trong X_test_processed: {np.isnan(X_test_processed).sum()}")
except Exception as e:
    # Bắt các lỗi khác có thể xảy ra
    print(f"Lỗi khi kiểm tra NaN: {e}")

Số NaN trong X_train_processed: 0
Số NaN trong X_test_processed: 0


In [49]:
if 'X_train_processed' in locals():
    knn_model = KNeighborsClassifier(n_neighbors=5) # Thử với k=5
    knn_model.fit(X_train_processed, y_train)
    print("Đã huấn luyện xong mô hình KNN.")
else:
    print("Bỏ qua huấn luyện KNN do thiếu dữ liệu đã xử lý.")
    knn_model = None

Đã huấn luyện xong mô hình KNN.


In [50]:
if knn_model is not None and 'X_test_processed' in locals():
    y_pred_knn = knn_model.predict(X_test_processed)

    print("--- Kết quả đánh giá KNN ---")
    print(f"Accuracy: {accuracy_score(y_test, y_pred_knn):.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_knn))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred_knn))
else:
    print("Bỏ qua đánh giá KNN.")


--- Kết quả đánh giá KNN ---
Accuracy: 0.3068

Classification Report:
                        precision    recall  f1-score   support

         Casual Driver       0.31      0.38      0.34        82
              Commuter       0.31      0.38      0.34        95
Long-Distance Traveler       0.29      0.16      0.21        87

              accuracy                           0.31       264
             macro avg       0.30      0.31      0.30       264
          weighted avg       0.30      0.31      0.30       264


Confusion Matrix:
[[31 38 13]
 [37 36 22]
 [32 41 14]]


In [51]:
if 'X_train_processed' in locals():
    dt_model = DecisionTreeClassifier(random_state=42) # random_state để kết quả ổn định
    dt_model.fit(X_train_processed, y_train)
    print("Đã huấn luyện xong mô hình Decision Tree.")
else:
    print("Bỏ qua huấn luyện Decision Tree do thiếu dữ liệu đã xử lý.")
    dt_model = None

Đã huấn luyện xong mô hình Decision Tree.


In [52]:
if dt_model is not None and 'X_test_processed' in locals():
    y_pred_dt = dt_model.predict(X_test_processed)

    print("--- Kết quả đánh giá Decision Tree ---")
    print(f"Accuracy: {accuracy_score(y_test, y_pred_dt):.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_dt))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred_dt))
else:
    print("Bỏ qua đánh giá Decision Tree.")

--- Kết quả đánh giá Decision Tree ---
Accuracy: 0.3371

Classification Report:
                        precision    recall  f1-score   support

         Casual Driver       0.34      0.40      0.37        82
              Commuter       0.32      0.32      0.32        95
Long-Distance Traveler       0.36      0.30      0.33        87

              accuracy                           0.34       264
             macro avg       0.34      0.34      0.34       264
          weighted avg       0.34      0.34      0.34       264


Confusion Matrix:
[[33 29 20]
 [39 30 26]
 [26 35 26]]


In [53]:
model_dir = 'models'
os.makedirs(model_dir, exist_ok=True) # exist_ok=True sẽ không báo lỗi nếu thư mục đã tồn tại

save_successful = True

# Lưu preprocessor
if preprocessor is not None:
    preprocessor_path = os.path.join(model_dir, 'preprocessor.joblib')
    try:
        joblib.dump(preprocessor, preprocessor_path)
        print(f"Đã lưu preprocessor vào: {preprocessor_path}")
    except Exception as e:
        print(f"Lỗi khi lưu preprocessor: {e}")
        save_successful = False
else:
    print("Không có preprocessor để lưu.")
    save_successful = False

# Lưu mô hình KNN
if knn_model is not None:
    knn_path = os.path.join(model_dir, 'knn_model.joblib')
    try:
        joblib.dump(knn_model, knn_path)
        print(f"Đã lưu mô hình KNN vào: {knn_path}")
    except Exception as e:
        print(f"Lỗi khi lưu mô hình KNN: {e}")
        save_successful = False
else:
    print("Không có mô hình KNN để lưu.")
    save_successful = False

# Lưu mô hình Decision Tree
if dt_model is not None:
    dt_path = os.path.join(model_dir, 'dt_model.joblib')
    try:
        joblib.dump(dt_model, dt_path)
        print(f"Đã lưu mô hình Decision Tree vào: {dt_path}")
    except Exception as e:
        print(f"Lỗi khi lưu mô hình Decision Tree: {e}")
        save_successful = False
else:
    print("Không có mô hình Decision Tree để lưu.")
    save_successful = False

if save_successful:
    print("\nHoàn tất! Preprocessor và các mô hình đã được lưu vào thư mục 'models'.")
else:
    print("\nCảnh báo: Có lỗi xảy ra trong quá trình lưu file.")


Đã lưu preprocessor vào: models\preprocessor.joblib
Đã lưu mô hình KNN vào: models\knn_model.joblib
Đã lưu mô hình Decision Tree vào: models\dt_model.joblib

Hoàn tất! Preprocessor và các mô hình đã được lưu vào thư mục 'models'.
